In [1]:
import numpy as np
import pandas as pd
import glob
import pickle
import cv2
import os
import re

In [2]:
# Function to extract DJI number from a file path
def extract_dji_number(file_path):
    match = re.search(r'DJI_(\d+)', file_path)
    return match.group(1) if match else None

In [3]:
DATE = ['20230308', '20230309', '20230311', '20230312']
SESSION = ['SM_Lek1', 'SE_Lek1']
DRONE = ['P1D1', 'P1D2', 'P2D3', 'P2D4', 'P3D5', 'P3D6']

registration_directory = '/Volumes/EAS_shared/blackbuck/working/processed/Field_Recording_2023/SpatialRegistration'
tracking_directory = '/Volumes/EAS_shared/blackbuck/working/processed/Field_Recording_2023/Tracking_Fusion'

In [4]:
missing_tracks = []  # To store missing tracks.csv paths

for date in DATE:
    for session in SESSION:
        for drone in DRONE:
            registration_path = f"{registration_directory}/{date}/{session}/{drone}"
            tracking_path = f"{tracking_directory}/{date}/{session}/{drone}/*{session}_{drone}*"

            # Get frames with corresponding anchors
            anchors = sorted(glob.glob(f"{registration_path}/{date}_{session}_{drone}*_Anchored.csv"))

            # Get tracking folders sorted by DJI video name
            tracking_folders = glob.glob(tracking_path)
            tracking_folders = sorted(tracking_folders, key=lambda x: int(re.search(r'DJI_(\d+)', x).group(1)))

            # Get homography matrices from frame to anchor frame
            homography_matrices = sorted(glob.glob(f"{registration_path}/{date}_{session}_{drone}*_homographies.pkl"))

            # Create dictionaries mapping DJI numbers to file paths
            anchors_dict = {extract_dji_number(file): file for file in anchors}
            tracking_dict = {extract_dji_number(file): file for file in tracking_folders}
            homography_dict = {extract_dji_number(file): file for file in homography_matrices}
            
            # Get sorted list of common DJI numbers
            common_dji_numbers = sorted(set(anchors_dict.keys()) & set(tracking_dict.keys()) & set(homography_dict.keys()), key=int)

            for dji in common_dji_numbers:
                print(date, session, drone, dji)

                try:
                    anchor_file = pd.read_csv(anchors_dict[dji])
                    tracks_csv_path = os.path.join(tracking_dict[dji], 'tracks.csv')
                    
                    if not os.path.isfile(tracks_csv_path):
                        raise FileNotFoundError(f"Missing file: {tracks_csv_path}")
                    
                    tracking_file = pd.read_csv(tracks_csv_path)

                    with open(homography_dict[dji], "rb") as f:
                        homography_file = pickle.load(f)

                    tracking_file = tracking_file.merge(anchor_file[['frame', 'best_anchor_frame']], on='frame', how='left')
                    tracking_file['x'] = tracking_file['bb_left'] + tracking_file['bb_width']/2
                    tracking_file['y'] = tracking_file['bb_top'] + tracking_file['bb_height']/2
                    tracking_file['idx'] = tracking_file['master_track_id']
                    tracking_file = tracking_file.drop_duplicates()

                    tracking_file = tracking_file.loc[:,['frame', 'x', 'y', 'idx', 'class_id', 'class_name', 'best_anchor_frame']]

                    for frame in tracking_file['frame'].unique():
                        if frame in homography_file:
                            H = homography_file[frame]
                            matched_points = tracking_file[tracking_file['frame'] == frame][['idx', 'x', 'y']]

                            if not matched_points.empty:
                                src_pts = np.array(matched_points[['x', 'y']], dtype=np.float32).reshape(-1, 1, 2)
                                transformed_pts = cv2.perspectiveTransform(src_pts, H)

                                tracking_file.loc[tracking_file['frame'] == frame, 'transformed_x'] = transformed_pts[:, 0, 0]
                                tracking_file.loc[tracking_file['frame'] == frame, 'transformed_y'] = transformed_pts[:, 0, 1]
                    
                    output_filename = os.path.basename(anchors_dict[dji]).replace('_Anchored.csv', '_Anchored_trajectories.csv')
                    tracking_file.to_csv(os.path.join(os.path.dirname(anchors_dict[dji]), output_filename), index=False, mode='w')

                except FileNotFoundError as e:
                    print(f"Warning: {e}")
                    missing_tracks.append(tracking_dict[dji])
                except Exception as e:
                    print(f"Error processing {date}, {session}, {drone}, {dji}: {e}")
                    continue

20230308 SM_Lek1 P1D1 0060
20230308 SM_Lek1 P1D1 0061
20230308 SM_Lek1 P1D1 0062
20230308 SM_Lek1 P1D1 0063
20230308 SM_Lek1 P1D1 0064
20230308 SM_Lek1 P1D1 0065
20230308 SM_Lek1 P1D1 0066
20230308 SM_Lek1 P1D1 0067
20230308 SM_Lek1 P1D1 0068
20230308 SM_Lek1 P1D1 0069
20230308 SM_Lek1 P1D1 0070
20230308 SM_Lek1 P1D1 0071
20230308 SM_Lek1 P1D1 0072
20230308 SM_Lek1 P1D1 0073
20230308 SM_Lek1 P1D1 0074
20230308 SM_Lek1 P1D1 0075
20230308 SM_Lek1 P1D1 0076
20230308 SM_Lek1 P1D1 0077
20230308 SM_Lek1 P1D1 0078
20230308 SM_Lek1 P1D1 0079
20230308 SM_Lek1 P1D1 0080
20230308 SM_Lek1 P1D1 0081
20230308 SM_Lek1 P1D1 0082
20230308 SM_Lek1 P1D1 0083
20230308 SM_Lek1 P1D1 0084
20230308 SM_Lek1 P1D1 0085
20230308 SM_Lek1 P1D1 0086
20230308 SM_Lek1 P1D1 0087
20230308 SM_Lek1 P1D2 0813
20230308 SM_Lek1 P1D2 0814
20230308 SM_Lek1 P1D2 0815
20230308 SM_Lek1 P1D2 0816
20230308 SM_Lek1 P1D2 0817
20230308 SM_Lek1 P1D2 0818
20230308 SM_Lek1 P1D2 0819
20230308 SM_Lek1 P1D2 0820
20230308 SM_Lek1 P1D2 0821
2

20230309 SE_Lek1 P1D2 0872
20230309 SE_Lek1 P1D2 0873
20230309 SE_Lek1 P1D2 0874
20230309 SE_Lek1 P1D2 0875
20230309 SE_Lek1 P1D2 0876
20230309 SE_Lek1 P1D2 0877
20230309 SE_Lek1 P1D2 0878
20230309 SE_Lek1 P1D2 0879
20230309 SE_Lek1 P1D2 0880
20230309 SE_Lek1 P1D2 0881
20230309 SE_Lek1 P1D2 0882
20230309 SE_Lek1 P1D2 0883
20230309 SE_Lek1 P1D2 0884
20230309 SE_Lek1 P1D2 0885
20230309 SE_Lek1 P1D2 0886
20230309 SE_Lek1 P1D2 0887
20230309 SE_Lek1 P1D2 0888
20230309 SE_Lek1 P1D2 0889
20230309 SE_Lek1 P1D2 0890
20230309 SE_Lek1 P1D2 0891
20230309 SE_Lek1 P1D2 0892
20230309 SE_Lek1 P1D2 0893
20230309 SE_Lek1 P1D2 0894
20230309 SE_Lek1 P2D3 0139
20230309 SE_Lek1 P2D3 0140
20230309 SE_Lek1 P2D3 0141
20230309 SE_Lek1 P2D3 0142
20230309 SE_Lek1 P2D3 0143
20230309 SE_Lek1 P2D3 0144
20230309 SE_Lek1 P2D3 0145
20230309 SE_Lek1 P2D3 0146
20230309 SE_Lek1 P2D3 0147
20230309 SE_Lek1 P2D3 0148
20230309 SE_Lek1 P2D3 0149
20230309 SE_Lek1 P2D3 0150
20230309 SE_Lek1 P2D3 0151
20230309 SE_Lek1 P2D3 0152
2

20230311 SE_Lek1 P2D3 0240
20230311 SE_Lek1 P2D3 0241
20230311 SE_Lek1 P2D3 0242
20230311 SE_Lek1 P2D3 0243
20230311 SE_Lek1 P2D3 0244
20230311 SE_Lek1 P2D3 0245
20230311 SE_Lek1 P2D3 0246
20230311 SE_Lek1 P2D3 0247
20230311 SE_Lek1 P2D3 0248
20230311 SE_Lek1 P2D3 0249
20230311 SE_Lek1 P2D3 0250
20230311 SE_Lek1 P2D3 0251
20230311 SE_Lek1 P2D3 0252
20230311 SE_Lek1 P2D3 0253
20230311 SE_Lek1 P2D3 0254
20230311 SE_Lek1 P2D3 0255
20230311 SE_Lek1 P2D3 0256
20230311 SE_Lek1 P2D3 0257
20230311 SE_Lek1 P2D3 0258
20230311 SE_Lek1 P2D3 0259
20230311 SE_Lek1 P2D3 0260
20230311 SE_Lek1 P2D3 0261
20230311 SE_Lek1 P2D3 0262
20230311 SE_Lek1 P2D4 0878
20230311 SE_Lek1 P2D4 0879
20230311 SE_Lek1 P2D4 0880
20230311 SE_Lek1 P2D4 0881
20230311 SE_Lek1 P2D4 0882
20230311 SE_Lek1 P2D4 0883
20230311 SE_Lek1 P2D4 0884
20230311 SE_Lek1 P2D4 0885
20230311 SE_Lek1 P2D4 0886
20230311 SE_Lek1 P2D4 0887
20230311 SE_Lek1 P2D4 0888
20230311 SE_Lek1 P2D4 0889
20230311 SE_Lek1 P2D4 0890
20230311 SE_Lek1 P2D4 0891
2

20230312 SE_Lek1 P2D4 0932
20230312 SE_Lek1 P2D4 0933
20230312 SE_Lek1 P2D4 0934
20230312 SE_Lek1 P2D4 0935
20230312 SE_Lek1 P2D4 0936
20230312 SE_Lek1 P2D4 0937
20230312 SE_Lek1 P3D5 0553
20230312 SE_Lek1 P3D5 0554
20230312 SE_Lek1 P3D5 0555
20230312 SE_Lek1 P3D5 0556
20230312 SE_Lek1 P3D5 0557
20230312 SE_Lek1 P3D5 0558
20230312 SE_Lek1 P3D5 0559
20230312 SE_Lek1 P3D5 0560
20230312 SE_Lek1 P3D5 0561
20230312 SE_Lek1 P3D5 0562
20230312 SE_Lek1 P3D5 0563
20230312 SE_Lek1 P3D5 0564
20230312 SE_Lek1 P3D5 0565
20230312 SE_Lek1 P3D5 0566
20230312 SE_Lek1 P3D5 0567
20230312 SE_Lek1 P3D5 0568
20230312 SE_Lek1 P3D5 0569
20230312 SE_Lek1 P3D5 0570
20230312 SE_Lek1 P3D5 0571
20230312 SE_Lek1 P3D5 0572
20230312 SE_Lek1 P3D6 0903
20230312 SE_Lek1 P3D6 0904
20230312 SE_Lek1 P3D6 0905
20230312 SE_Lek1 P3D6 0906
20230312 SE_Lek1 P3D6 0907
20230312 SE_Lek1 P3D6 0908
20230312 SE_Lek1 P3D6 0909
20230312 SE_Lek1 P3D6 0910
20230312 SE_Lek1 P3D6 0911
20230312 SE_Lek1 P3D6 0912
20230312 SE_Lek1 P3D6 0913
2